# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
enable_visualization = False
if enable_visualization:
    env = UnityEnvironment(file_name='Tennis_Linux/Tennis.x86_64')
else:
    env = UnityEnvironment(file_name='Tennis_Linux_NoVis/Tennis.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    num_iter_for_one_game = 0
    while True:
        num_iter_for_one_game += 1
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            print('game done in ',num_iter_for_one_game,' iters')
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

game done in  15  iters
Score (max over agents) from episode 1: 0.0
game done in  15  iters
Score (max over agents) from episode 2: 0.0
game done in  14  iters
Score (max over agents) from episode 3: 0.0
game done in  15  iters
Score (max over agents) from episode 4: 0.0
game done in  14  iters
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
from collections import deque
from ddpg_agent import Agent
from matplotlib import pyplot as plt
import os
import datetime
import torch

# widget bar to display progress
# !pip install progressbar
import progressbar as pb


def GetFolderPath(name='result'):
    cur_folder = os.getcwd()
    str_time = datetime.datetime.strftime(datetime.datetime.now(),
                                          '%Y_%m_%d_%H_%M_%S')
    folder_name = '/' + name + '_' + str_time + '/'
    save_path = cur_folder + folder_name
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    return save_path


def plot_result(scores, scores_avg, actual_target_score, save_path):
    target_score_curve = np.ones(len(scores)) * actual_target_score
    fig = plt.figure(figsize=[15,10])
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    plt.xlabel("episode"), plt.ylabel("score")
    ax.plot(scores)
    ax.plot(scores_avg)
    ax.plot(target_score_curve)
    ax.legend(['score','avg_score', 'target_score'])
    plt.savefig(save_path)
    plt.close(fig)

# Try 1
# Original settings(ddpg_result_2023_05_21_18_19_39): scores not increasing
# Try 2
# Change BATCH_SIZE 128->1000(ddpg_result_2023_05_21_18_38_24): scores increase a little(0.1) from time to time
# Try 3
# Above and Change episode 300->1000 BATCH_SIZE to 40(ddpg_result_2023_05_21_19_12_01): not working
# Try 4
# Above and Change episode 300->1000 BATCH_SIZE to 40 and UPDATE_TIMES 1->10(ddpg_result_2023_05_21_19_20_03): succeeded in 550 episodes
# Try 5:
# Change model param save logic and episode 1000->1500(ddpg_result_2023_05_21_22_08_57): succeeded in 1500 episodes


def Learn(env: UnityEnvironment, n_episodes=1000, max_t=1000, target_score=0.0, actual_target_score=0.0, prioritized_learn=False):

    save_path = GetFolderPath('ddpg_result')

    widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
    timer = pb.ProgressBar(widgets=widget, maxval=n_episodes).start()

    scores_window = deque(maxlen=100)
    max_agent_scores = []
    avg_scores = []
    cur_target_score = target_score 
    target_score_reached_counter = 0
    target_score_reached_num_needed = int(n_episodes * 0.20)

    agent = Agent(env, save_path+'log.txt')
    
    for i_episode in range(1, n_episodes+1):
        agent.reset()
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        num_agents = len(env_info.agents)
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        for t in range(max_t):
            actions = agent.multi_agent_act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            # agent.multi_agent_step(states, actions, rewards, next_states, dones)
            for i in range(num_agents):
                if prioritized_learn:
                    agent.priority_step(states[i], actions[i], rewards[i], next_states[i], dones[i], t)
                else:
                    agent.step(states[i], actions[i], rewards[i], next_states[i], dones[i], t)
            states = next_states                               # roll over states to next time step
            scores += env_info.rewards                         # update the score (for each agent)
            # print('current time: ', t)
            if np.any(dones):                                  # exit loop if episode finished
                break
        # we take the max score of two agents as score for each episode
        scores_window.append(np.max(scores))
        print('\nEpisode {}\tTotal score (max over agents) this episode: {}'.format(i_episode, scores_window[-1]))
        max_agent_scores.append(scores_window[-1])
        avg_scores.append(np.mean(scores_window))
        
        # update progress widget bar
        timer.update(i_episode)
        
        model_saved = False
        result_str = 'epi_' + str(i_episode) + '_score_' + str(avg_scores[-1])
        if max_agent_scores[-1] > cur_target_score:
            print('\nEnvironment improved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, avg_scores[-1]))
            torch.save(agent.critic_local.state_dict(), save_path+result_str+'_critic_checkpoint.pth')
            torch.save(agent.actor_local.state_dict(), save_path+result_str+'_actor_checkpoint.pth')
            cur_target_score = max_agent_scores[-1]
            model_saved = True
        
        pic_save_path = save_path+'results/'
        if not os.path.isdir(pic_save_path):
            os.mkdir(pic_save_path)
        pic_name = 'ddpg_n_epi_'+str(i_episode)+'.png'    
        plot_result(max_agent_scores, avg_scores, actual_target_score, pic_save_path+pic_name) 
        
        if i_episode > 100 and avg_scores[-1] > actual_target_score:
            print('training done')
            target_score_reached_counter += 1
            if not model_saved:
                torch.save(agent.critic_local.state_dict(), save_path+result_str+'_critic_checkpoint.pth')
                torch.save(agent.actor_local.state_dict(), save_path+result_str+'_actor_checkpoint.pth')
            if target_score_reached_counter >= target_score_reached_num_needed:
                break
    
    timer.finish()

    return avg_scores

In [ ]:
# ddpg
prioritized_learn = False
scores = Learn(env, 1500, 10000, 0.0, 0.5, prioritized_learn)

In [8]:
# ddpg with prioritized experience replay
prioritized_learn = True
scores = Learn(env, 1500, 10000, 0.0, 0.5, prioritized_learn)

training loop:   0% |                                          | ETA:  --:--:--
training loop:   0% |                                          | ETA:   0:03:57


state size: 24 action size: 2

Episode 1	Total score (max over agents) this episode: 0.0


training loop:   0% |                                          | ETA:   0:09:25



Episode 2	Total score (max over agents) this episode: 0.0


training loop:   0% |                                          | ETA:   0:09:46


...

training loop:  99% |######################################### | ETA:   0:00:10


t_step: 25 	, sample_time: 0.004 	update_time: 0.006 	learn_time: 0.018 	sum: 4271.985 	size: 230586                        
Episode 1499	Total score (max over agents) this episode: 0.09000000171363354
t_step: 80 	, sample_time: 0.003 	update_time: 0.004 	learn_time: 0.012 	sum: 4282.160 	size: 230753                        

training loop: 100% |##########################################| ETA:  00:00:00


t_step: 80 	, sample_time: 0.003 	update_time: 0.005 	learn_time: 0.013 	sum: 4264.444 	size: 230754                        
Episode 1500	Total score (max over agents) this episode: 0.19000000320374966


training loop: 100% |##########################################| Time:  4:24:04
